In [ ]:
import logging
import matplotlib.pyplot as plt
import nni
import numpy as np
import torch
import torch.optim as optim
import torch.nn.functional as F
import nni.retiarii.nn.pytorch as nn
import nni.retiarii.strategy as strategy

from collections import OrderedDict

from nni.experiment import Experiment
from nni.retiarii import model_wrapper
from nni.retiarii.evaluator import FunctionalEvaluator
from nni.retiarii.experiment.pytorch import RetiariiExperiment, RetiariiExeConfig

from skimage.metrics import peak_signal_noise_ratio as skimage_psnr

from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from torchvision.datasets import MNIST
from torchvision import datasets, transforms


logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def psnr(image_true, image_test):
    # Convert PyTorch tensors to NumPy arrays
    if torch.is_tensor(image_true):
        image_true = image_true.detach().cpu().numpy()
    if torch.is_tensor(image_test):
        image_test = image_test.detach().cpu().numpy()
    return skimage_psnr(image_true, image_test)

def add_noise(img, noise_factor=0.5):
    """Add random noise to an image."""
    noise = torch.randn_like(img) * noise_factor
    noisy_img = img + noise
    return torch.clamp(noisy_img, 0., 1.)

def deep_image_prior_denoising(model, noisy_img, clean_img, device, optimizer, iterations=250):
    model.train()
    for iteration in range(iterations):
        optimizer.zero_grad()
        output = model(torch.randn(noisy_img.shape).to(device))
        loss = nn.MSELoss()(output, noisy_img)
        loss.backward()
        optimizer.step()
        if iteration % 25 == 0:
            # Calculate PSNR
            with torch.no_grad():
                denoised_output = model(noisy_img)
                psnr_value = psnr(clean_img, denoised_output)
            print('Iteration: {}\tLoss: {:.6f}\tPSNR: {:.6f} dB'.format(iteration, loss.item(), psnr_value))
            nni.report_intermediate_result(psnr_value)
    return output

def evaluate_denoising(denoised_img, clean_img):
    # We no longer need the model in an eval state or any forward pass here
    # because the denoised image is already generated and passed to the function.
    return psnr(clean_img, denoised_img)

def main_evaluation(model_cls):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    
    # Instantiate model
    model = model_cls().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    transform = transforms.Compose([transforms.ToTensor()])
    dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    img, _ = dataset[0]  # Original clean image
    noisy_img = add_noise(img).unsqueeze(0).to(device)  # Noisy version of image

    # Denoise the image for a set number of iterations
    denoised_img = deep_image_prior_denoising(model, noisy_img, img.unsqueeze(0).to(device), device, optimizer)

    # Evaluate the PSNR of the denoised image
    psnr_value = evaluate_denoising(denoised_img, img.unsqueeze(0).to(device))
    print('PSNR: {:.6f} dB'.format(psnr_value))

    # Report final PSNR to NNI
    nni.report_final_result(psnr_value.item())

# base

In [ ]:
class Convolutions(nn.Module):
    def __init__(self, out_channels, activations, convs1, convs2, layer_name):
        super().__init__()

        self.conv1 = nn.LayerChoice(convs1, label=f'{layer_name} - Step 1: Convolution 1')
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.act1 = nn.LayerChoice(activations, label=f'{layer_name} - Step 2: Activation 1')
        
        self.conv2 = nn.LayerChoice(convs2, label=f'{layer_name} - Step 3: Convolution 2')
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.act2 = nn.LayerChoice(activations, label=f'{layer_name} - Step 4: Activation 2')

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.act2(x)
        return x
    
class BaseBlock(nn.Module):
    def __init__(self):
        super(BaseBlock, self).__init__()

    def get_conv_ordered_dict(self, in_channels, out_channels, ks, pd, dl, first=True):
        layers = [
            ("Conv2d", nn.Conv2d(in_channels, out_channels, kernel_size=ks, padding=pd, dilation=dl)),
            ("DepthwiseSeparable", nn.Sequential(
                nn.Conv2d(in_channels, in_channels, kernel_size=ks, padding=pd, dilation=dl, groups=in_channels),
                nn.Conv2d(in_channels, out_channels, kernel_size=1)
                )
            )
        ]
        if not first:
            layers.append(("Depthwise", nn.Conv2d(in_channels, out_channels, kernel_size=1)))
        return OrderedDict(layers)

    def crop_tensor(self, target_tensor, tensor):
        target_size = target_tensor.size()[2]  # Assuming height and width are same
        tensor_size = tensor.size()[2]
        delta = tensor_size - target_size
        delta = delta // 2
        return tensor[:, :, delta:tensor_size-delta, delta:tensor_size-delta]

class EncoderBlock(BaseBlock):
    def __init__(self, in_channels, out_channels, ks, pd, dl, activations, downsamples, layer_name):
        super(EncoderBlock, self).__init__()
        
        self.downsample = nn.LayerChoice(downsamples,label=f'{layer_name} - Step 0: Downsampling Technique')
        self.conv_layer = Convolutions(out_channels, 
                                       activations, 
                                       self.get_conv_ordered_dict(in_channels, out_channels, ks, pd, dl),
                                       self.get_conv_ordered_dict(out_channels, out_channels, ks, pd, dl, first=False), 
                                       layer_name)

    def forward(self, x):
        x = self.downsample(x)
        x = self.conv_layer(x)
        return x


class DecoderBlock(BaseBlock):
    def __init__(self, in_channels, out_channels, ks, pd, dl, activations, upsamples, layer_name):
        super(DecoderBlock, self).__init__()

        self.upsample = nn.LayerChoice(upsamples, label=f"{layer_name} - Step 0: Upsampling Technique")
        self.conv_layer = Convolutions(out_channels, 
                                       activations, 
                                       self.get_conv_ordered_dict(in_channels, out_channels, ks, pd, dl),
                                       self.get_conv_ordered_dict(out_channels, out_channels, ks, pd, dl, first=False), 
                                       layer_name)

    def forward(self, x, skip):
        upsampled = self.upsample(x)
        cropped = self.crop_tensor(upsampled, skip)
        return self.conv_layer(torch.cat([cropped, upsampled], 1))

@model_wrapper
class SearchSpace(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super().__init__()

        ks = nn.ValueChoice([1, 3, 5, 7, 9], label="Kernel Size")
        dl = nn.ValueChoice([1, 2, 3, 4, 5], label="Dilation Rate")
        pd = (ks - 1) * dl // 2
        # pdd = ks // 2

        activations = OrderedDict([
            ("RelU", nn.ReLU(inplace=True)),
            ("LeakyRelU", nn.LeakyReLU(inplace=True)),
            ("Sigmoid", nn.Sigmoid()),
            ("Selu", nn.SELU(inplace=True)),
            ("PreLU", nn.PReLU()),
            ("SiLU", nn.SiLU(inplace=True)),
        ])

        downsamples = OrderedDict([
            ("AvgPool2d", nn.AvgPool2d(kernel_size=2, stride=2)),
            ("MaxPool2d", nn.MaxPool2d(kernel_size=2, stride=2)),
        ])

        upsamples = OrderedDict([
            ("Nearest", nn.Upsample(scale_factor=2,mode='nearest')),
            ("Bilinear", nn.Upsample(scale_factor=2,mode='bilinear', align_corners=True)),
            ("Bicubic", nn.Upsample(scale_factor=2,mode='bicubic', align_corners=True))
        ])

        # Conv layer in"
        self.layer1_out_channels = 64
        self.convs1 = self.get_conv_ordered_dict(in_channels, self.layer1_out_channels, ks, pd, dl)
        self.convs2 = self.get_conv_ordered_dict(self.layer1_out_channels, self.layer1_out_channels, ks, pd, dl, first=False)
        self.first = Convolutions(self.layer1_out_channels, activations, self.convs1, self.convs2, "First Conv Layer")

        # Encoders
        self.Encoder1 = EncoderBlock(64,   128, ks, pd, dl, activations, downsamples, "Encoder 1")
        self.Encoder2 = EncoderBlock(128,  256, ks, pd, dl, activations, downsamples, "Encoder 2")
        self.Encoder3 = EncoderBlock(256,  512, ks, pd, dl, activations, downsamples, "Encoder 3")
        self.Encoder4 = EncoderBlock(512, 1024, ks, pd, dl, activations, downsamples, "Encoder 4")

        # Decoders
        self.Decoder1 = DecoderBlock(512*3, 512, ks, pd, dl, activations, upsamples, "Decoder 1")
        self.Decoder2 = DecoderBlock(256*3,  256, ks, pd, dl, activations, upsamples, "Decoder 2")
        self.Decoder3 = DecoderBlock(128*3,  128, ks, pd, dl, activations, upsamples, "Decoder 3")
        self.Decoder4 = DecoderBlock(64*3,    64, ks, pd, dl, activations, upsamples, "Decoder 4")

        # Conv layer out
        self.out = nn.Conv2d(64, out_channels, kernel_size=ks, padding=pd, dilation=dl)

    def forward(self, x):
        logger.info("Input: %s", x.size())
        
        encoders = [self.first, self.Encoder1, self.Encoder2, self.Encoder3, self.Encoder4]
        decoders = [self.Decoder1, self.Decoder2, self.Decoder3, self.Decoder4]
        
        # Variables to store intermediate values
        encoder_outputs = []

        # Encoder pass
        for i, encoder in enumerate(encoders):
            x = encoder(x)
            encoder_outputs.append(x)
            logger.info(f"Encoder {i if i != 0 else 'Conv layer in'}: %s", x.size())

        # Decoder pass
        for i, decoder in enumerate(decoders):
            x = decoder(x, encoder_outputs[-(i+2)])
            logger.info(f"Decoder {4-i}: %s", x.size())

        x = self.out(x)
        logger.info("Output: %s", x.size())
        return x
    
    def get_conv_ordered_dict(self, in_channels, out_channels, ks, pd, dl, first=False):
         layers = [
                ("Conv2d", nn.Conv2d(in_channels, out_channels, kernel_size=ks, padding=pd, dilation=dl)),
                ("DepthwiseSeparable", nn.Sequential(
                    nn.Conv2d(in_channels, in_channels, kernel_size=ks, padding=pd, dilation=dl, groups=in_channels),
                    nn.Conv2d(in_channels, out_channels, kernel_size=1, padding=0, dilation=1)
                    )
                )
            ]
         if not first:
            layers.append(("Depthwise", nn.Conv2d(in_channels, out_channels, kernel_size=1, padding=0, dilation=1)))
         return OrderedDict(layers)



# new

In [ ]:
class Convolutions(nn.Module):
    def __init__(self, out_channels, activations, convs1, convs2, layer_name):
        super().__init__()

        self.conv1 = nn.LayerChoice(convs1, label=f'{layer_name} - Step 1: Convolution 1')
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.act1 = nn.LayerChoice(activations, label=f'{layer_name} - Step 2: Activation 1')
        
        self.conv2 = nn.LayerChoice(convs2, label=f'{layer_name} - Step 3: Convolution 2')
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.act2 = nn.LayerChoice(activations, label=f'{layer_name} - Step 4: Activation 2')

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.act2(x)
        return x
    
class BaseBlock(nn.Module):
    def __init__(self):
        super(BaseBlock, self).__init__()

    def get_conv_ordered_dict(self, in_channels, out_channels, ks, pd, dl, first=True):
        layers = [
            ("Conv2d", nn.Conv2d(in_channels, out_channels, kernel_size=ks, padding=pd, dilation=dl)),
            ("DepthwiseSeparable", nn.Sequential(
                nn.Conv2d(in_channels, in_channels, kernel_size=ks, padding=pd, dilation=dl, groups=in_channels),
                nn.Conv2d(in_channels, out_channels, kernel_size=1)
                )
            )
        ]
        if not first:
            layers.append(("Depthwise", nn.Conv2d(in_channels, out_channels, kernel_size=1)))
        return OrderedDict(layers)

    def crop_tensor(self, target_tensor, tensor):
        target_size = target_tensor.size()[2]  # Assuming height and width are same
        tensor_size = tensor.size()[2]
        delta = tensor_size - target_size
        delta = delta // 2
        return tensor[:, :, delta:tensor_size-delta, delta:tensor_size-delta]

class EncoderBlock(BaseBlock):
    def __init__(self, in_channels, out_channels, ks, pd, dl, activations, downsamples, layer_name):
        super(EncoderBlock, self).__init__()
        
        self.downsample = nn.LayerChoice(downsamples,label=f'{layer_name} - Step 0: Downsampling Technique')
        self.conv_layer = Convolutions(out_channels, 
                                       activations, 
                                       self.get_conv_ordered_dict(in_channels, out_channels, ks, pd, dl),
                                       self.get_conv_ordered_dict(out_channels, out_channels, ks, pd, dl, first=False), 
                                       layer_name)

    def forward(self, x):
        x = self.downsample(x)
        x = self.conv_layer(x)
        return x


class DecoderBlock(BaseBlock):
    def __init__(self, in_channels, out_channels, ks, pd, dl, activations, upsamples, layer_name):
        super(DecoderBlock, self).__init__()

        self.upsample = nn.LayerChoice(upsamples, label=f"{layer_name} - Step 0: Upsampling Technique")
        self.conv_layer = Convolutions(out_channels, 
                                       activations, 
                                       self.get_conv_ordered_dict(in_channels, out_channels, ks, pd, dl),
                                       self.get_conv_ordered_dict(out_channels, out_channels, ks, pd, dl, first=False), 
                                       layer_name)

    def forward(self, x, skip):
        upsampled = self.upsample(x)
        cropped = self.crop_tensor(upsampled, skip)
        return self.conv_layer(torch.cat([cropped, upsampled], 1))

@model_wrapper
class SearchSpace(BaseBlock):
    def __init__(self, in_channels=3, out_channels=3):
        super().__init__()

        network_depth = nn.ValueChoice([1, 2, 3, 4], label="Network Depth")
        # maybe can use max of depth to create a better label down in the decoder_block = ... line below

        ks = nn.ValueChoice([1, 3, 5, 7, 9], label="Kernel Size")
        dl = nn.ValueChoice([1, 2, 3, 4, 5], label="Dilation Rate")
        pd = (ks - 1) * dl // 2

        activations = OrderedDict([
            ("RelU", nn.ReLU(inplace=True)),
            ("LeakyRelU", nn.LeakyReLU(inplace=True)),
            ("Sigmoid", nn.Sigmoid()),
            ("Selu", nn.SELU(inplace=True)),
            ("PreLU", nn.PReLU()),
            ("SiLU", nn.SiLU(inplace=True)),
        ])

        downsamples = OrderedDict([
            ("AvgPool2d", nn.AvgPool2d(kernel_size=2, stride=2)),
            ("MaxPool2d", nn.MaxPool2d(kernel_size=2, stride=2)),
        ])

        upsamples = OrderedDict([
            ("Nearest", nn.Upsample(scale_factor=2,mode='nearest')),
            ("Bilinear", nn.Upsample(scale_factor=2,mode='bilinear', align_corners=True)),
            ("Bicubic", nn.Upsample(scale_factor=2,mode='bicubic', align_corners=True))
        ])

        # Conv layer in"
        self.mid_channels = 64
        self.convs1 = self.get_conv_ordered_dict(in_channels, self.mid_channels, ks, pd, dl)
        self.convs2 = self.get_conv_ordered_dict(self.mid_channels, self.mid_channels, ks, pd, dl, first=False)
        self.first = Convolutions(self.mid_channels, activations, self.convs1, self.convs2, "First Conv Layer")


        # For Encoders:
        encoder_block = lambda index: EncoderBlock(64*(2**index), 64*(2**(index+1)), ks, pd, dl, activations, downsamples, f"Encoder {index+1}")
        self.encoders = nn.Repeat(encoder_block, network_depth)

        # For Decoders:
        decoder_block = lambda index: DecoderBlock(64*(2**(index))*3, 64*(2**index), ks, pd, dl, activations, upsamples, f"Decoder {index+1}")
        self.decoders = nn.Repeat(decoder_block, network_depth)
        self.decoders = self.decoders[::-1]

        # Conv layer out
        self.out = nn.Conv2d(64, out_channels, kernel_size=ks, padding=pd, dilation=dl)
        
    def forward(self, x):
        logger.info("Input: %s", x.size())
        
        # Variables to store intermediate values
        encoder_outputs = []

        # Start with the first conv layer
        x = self.first(x)
        encoder_outputs.append(x)
        logger.info(f"Initial Conv Layer: %s", x.size())

        # Encoder pass
        for i, encoder in enumerate(self.encoders):
            x = encoder(x)
            encoder_outputs.append(x)
            logger.info(f"Encoder {i+1}: %s", x.size())

        # Decoder pass
        for i, decoder in enumerate(self.decoders):
            x = decoder(x, encoder_outputs[-(i+2)])
            logger.info(f"Decoder {len(self.decoders) - i}: %s", x.size())

        x = self.out(x)
        logger.info("Output: %s", x.size())
        return x


In [ ]:
# search space
model_space = SearchSpace()
evaluator = FunctionalEvaluator(main_evaluation)

# search strategy
search_strategy = strategy.Random(dedup=True)

# experiment
exp = RetiariiExperiment(model_space, evaluator, [], search_strategy)
exp_config = RetiariiExeConfig('local')
exp_config.experiment_name = 'mnist_search'
exp_config.trial_code_directory = 'C:/Users/Public/Public_VS_Code/NAS_test'
exp_config.experiment_working_directory = 'C:/Users/Public/nni-experiments'

exp_config.max_trial_number = 50   # spawn 50 trials at most
exp_config.trial_concurrency = 2  # will run two trials concurrently

exp_config.trial_gpu_number = 1 # will run 1 trial(s) concurrently
exp_config.training_service.use_active_gpu = True

# Execute
exp.run(exp_config, 8081)

In [ ]:
experiment = Experiment.connect(8081)
experiment.stop()